In [1]:
import os
from tqdm.auto import tqdm
from dotenv import load_dotenv
import pinecone
from pinecone import ServerlessSpec
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from pinecone_text.sparse import BM25Encoder
from langchain_community.retrievers import PineconeHybridSearchRetriever
from langchain_cohere import CohereRerank
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
from pinecone import Pinecone
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# Modern LangChain imports
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

/home/rohan/.pyenv/versions/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
print("Loading and splitting documents...")
loader = PyPDFLoader("/home/rohan/S5/projdump/Carbon_adsorption.pdf")
documents = loader.load()

Loading and splitting documents...


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
print(f"Loaded and split {len(docs)} chunks.")

Loaded and split 41 chunks.


In [ ]:
# --- 2. Initialize Pinecone and Create Index ---
print("Initializing Pinecone...")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
if not pinecone_api_key:
    raise ValueError("PINECONE_API_KEY not found in environment variables.")

index_name = "langchain-test-index"  # change if desired
pc = Pinecone(api_key=pinecone_api_key)
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

Initializing Pinecone...


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'db9b807c3cbb38f438fe8386e0e4af5a', 'date': 'Sat, 23 Aug 2025 10:16:05 GMT', 'server': 'Google Frontend', 'Content-Length': '200', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"INVALID_ARGUMENT","message":"Bad request: Your free plan does not support indexes in the eu-west-1 region of aws. To create indexes in this region, upgrade your plan."},"status":400}


In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
gemini_api_key = os.getenv("GEMINI_API_KEY")
print("Loading and splitting documents...")
loader = PyPDFLoader("/home/rohan/S5/projdump/Carbon_adsorption.pdf")
documents = loader.load()
# embeddings = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001",
#     google_api_key=gemini_api_key,
#     max_retries=3,
#     timeout=60,
# )

from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(model="embed-english-v3.0")
# embeddings = HuggingFaceEmbeddings(model_name="/home/rohan/models")
bm25_encoder = BM25Encoder().default()
print("Embeddings and sparse encoder initialized.")

Loading and splitting documents...
Embeddings and sparse encoder initialized.


In [7]:
# how many chunks produced locally
print("Num doc chunks:", len(docs))

# pinecone index stats (replace `index` with your Pinecone index object if different)
print("Index stats:", index.describe_index_stats())


Num doc chunks: 41
Index stats: {'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [8]:
# --- 4. Upsert Documents to Pinecone (Hybrid Search) ---
print("Upserting documents to Pinecone with hybrid search...")
batch_size = 100
for i in tqdm(range(0, len(docs), batch_size)):
    i_end = min(i + batch_size, len(docs))
    docs_batch = docs[i:i_end]
    
    # Get page content
    docs_content = [doc.page_content for doc in docs_batch]

    # Create sparse vectors
    sparse_embeds = bm25_encoder.encode_documents(docs_content)

    # Create dense vectors
    dense_embeds = embeddings.embed_documents(docs_content)

    # Create metadata
    metadata = [
        {
            "text": doc.page_content,
            "source": doc.metadata.get("source", "Unknown"),
            "page": doc.metadata.get("page", 0),
        }
        for doc in docs_batch
    ]

    # Create IDs
    ids = [f"doc_{i+j}" for j in range(len(docs_batch))]
    
    # Create vectors for upsert
    vectors_to_upsert = []
    for doc_id, sparse, dense, meta in zip(ids, sparse_embeds, dense_embeds, metadata):
        vectors_to_upsert.append({
            "id": doc_id,
            "sparse_values": sparse,
            "values": dense,
            "metadata": meta,
        })

    # Upsert to Pinecone
    index.upsert(vectors=vectors_to_upsert)

print("Documents upserted to Pinecone.")

Upserting documents to Pinecone with hybrid search...


100%|██████████| 1/1 [00:03<00:00,  3.90s/it]

Documents upserted to Pinecone.


In [9]:
# --- 5. Initialize Retriever with Hybrid Search ---
print("Initializing retriever with hybrid search...")
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings,
    parse_encoder=bm25_encoder, 
    index=index, text_key = "text", 
    top_k=20
)
print("Retriever initialized.")

Initializing retriever with hybrid search...
Retriever initialized.


In [10]:
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model = "llama-3.3-70b-versatile",
    temperature = 0.2,
    max_tokens = 32000
)


In [11]:
# --- 6. Initialize Reranker ---
print("Initializing Cohere reranker...")
cohere_api_key = os.getenv("COHERE_API_KEY")
if not cohere_api_key:
    raise ValueError("COHERE_API_KEY not found in environment variables.")
reranker = CohereRerank(model="rerank-english-v3.0")
print("Reranker initialized.")

Initializing Cohere reranker...
Reranker initialized.


In [12]:
# --- 7. Create Conversational RAG Chain ---
print("Creating Conversational RAG Chain...")

# Use the default chain without custom prompt to avoid validation errors
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    condense_question_llm=llm,
    return_source_documents=True
)
print("Conversational RAG Chain created successfully.")

Creating Conversational RAG Chain...
Conversational RAG Chain created successfully.


In [ ]:
# Alternative: Creating ConversationalRetrievalChain with Custom Prompt
# Updated to use modern LangChain approach instead of deprecated load_qa_chain

from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

# Create a custom prompt template for the document chain
custom_prompt_template = """You are an enterprise-grade RAG agent designed to answer questions based on provided context.
Q4_K_M 
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {input}

Answer: """

# Create the prompt using ChatPromptTemplate (modern approach)
custom_prompt = ChatPromptTemplate.from_template(custom_prompt_template)

# Create the document chain with custom prompt using the new approach
doc_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=custom_prompt
)

# Create the retrieval chain (replaces ConversationalRetrievalChain for basic Q&A)
qa_chain_custom = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=doc_chain
)

print("Modern RAG chain with custom prompt created successfully.")

Modern RAG chain with custom prompt created successfully.


In [16]:
# --- 8. Run the RAG Agent ---
print("\n--- RAG Agent is Ready ---")

# For basic Q&A without conversation history, use the modern chain
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    
    # Use the modern chain (qa_chain_custom or qa_chain)
    result = qa_chain_custom.invoke({"input": query})
    
    # Rerank the retrieved documents
    reranked_docs = reranker.compress_documents(
        documents=result['context'],  # Changed from 'source_documents' to 'context'
        query=query
    )
    
    print("\n--- Answer ---")
    print(result['answer'])
    print("\n--- Reranked Sources ---")
    for doc in reranked_docs:
        print(f"Source: {doc.metadata['source']}, Page: {doc.metadata['page']}")
        print(f"Content: {doc.page_content[:200]}...")
        print("-" * 20)


--- RAG Agent is Ready ---

--- Answer ---
Here is a summary of the document in 100 lines or less:

The document discusses various studies on CO2 capture and adsorption using different materials.
These materials include carbon foams, biochars, and zeolitic imidazolate frameworks (ZIFs).
The studies investigate the effects of temperature, pressure, and surface area on CO2 adsorption.
Some materials show high CO2 adsorption capacities, such as 6.81 mmol/g at 273 K.
Others exhibit excellent selectivity, with CO2:N2 ratios of up to 21.
The document highlights the importance of optimizing material properties for CO2 capture.
This includes adjusting pore size, surface area, and functional groups to enhance adsorption.
The studies use various characterization techniques, such as BET, XPS, and FTIR.
They also employ different synthesis methods, including pyrolysis, hydrothermal treatment, and wet functionalization.
The goal of these studies is to develop efficient and selective CO2 adsorbents

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 23 Aug 2025 04:42:51 GMT', 'Content-Type': 'application/json', 'Content-Length': '81', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '314', 'x-pinecone-request-id': '7474914773618966712', 'x-envoy-upstream-service-time': '32', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Sparse vector must contain at least one value","details":[]}


In [ ]:
if pc.has_index(index_name):
    pc.delete_index(index_name)

In [ ]:
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